# Business Understanding

Our company has decided to create a **new movie studio**. This is a new venture for the company and our first introduction to the film industry. I will be exploring what types of films are currently doing the best at the box office. 

In order to help decide what types of films to create, I investigate the following 3 business questions:
1. What **genre** movie should we produce?
2. Who should **direct** the movie?
3. What **movie rating** should the movie be?
4. ##TODO #3 In which **month** should the movies be **released**?

# Data Understanding

I will be using 3 datasets in this notebook.
1. **bom.movie_gross.csv.gz**: This is a dataset from [BoxOfficeMojo](https://www.BoxOfficeMojo.com) containing domestic and foreign gross of 3,387 movies. I use this dataset to to determine the most profitable movies.
2. **im.db**: This is a database from [IMDB](https://www.IMDB.com) containing 8 tables. I chose 3 tables to use - movie_basics, directors, and persons.  `movie_basics` has basic movie information for 146,144 movies. `directors` has 140,417 entries of movie id's matched to person id's. `persons` has 606648 people related to the movies with their birth year, death year, and primary professions. I use this dataset to determine the director and genre that will result in the most profitable movie.
3. **rt.reviews.tsv.gz**: This is a dataset from [Rotten Tomatoes](https://www.RottenTomatoes.com) containing ratings and additional movie information for 1,560 movies. I use this dataset to determine the movie rating that will result in the most profitable movie.

In [66]:
# Import necessary libraries
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.options.mode.copy_on_write = True

In [2]:
# Save Box Office Mojo dataset to bom_df
bom_df = pd.read_csv('data/zippedData/bom.movie_gross.csv.gz')

In [135]:
# Save The Numbers dataset to budget_df
budget_df = pd.read_csv('data/zippedData/tn.movie_budgets.csv.gz')

In [3]:
# Save IMDB dataset to conn
conn = sqlite3.connect('data/zippedData/im.db')

# Save movie_basics table as a Pandas dataframe movie_basics_df
query_movie_basics = """

SELECT *
  FROM movie_basics
"""

movie_basics_df = pd.read_sql(query_movie_basics, conn)

# Save directors table as a Pandas dataframe directors_df
query_directors = """

  SELECT *
    FROM directors
GROUP BY movie_id
"""

directors_df = pd.read_sql(query_directors, conn)


movie_basics_df = pd.read_sql(query_movie_basics, conn)

# Save persons table as a Pandas dataframe persons_df
query_persons = """

SELECT *
  FROM persons
"""

persons_df = pd.read_sql(query_persons, conn)

In [4]:
# Save Rotten Tomatoes dataset to rt_movie_info_df
rt_movie_info_df = pd.read_csv('data/zippedData/rt.movie_info.tsv.gz', sep='\t', encoding='windows-1252')

## Data Preparation

I take a closer look at each dataset in this section. 

### Box Office Mojo

In [5]:
# Preview first 5 entries of bom_df
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


I take a look at the columns, number of nulls, and data types. I notice `foreign_gross` is type object. I will have to convert this data to floats when clearning. 

In [6]:
# View the overall shape, dtypes and null counts for each column in bom_df
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### The Numbers

In [136]:
# Preview first 5 entries of budget_df
budget_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [137]:
# View the overall shape, dtypes and null counts for each column in bom_df
budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


There are not 

### IMDB

In [7]:
# Preview first 5 entries of movie_basics_df
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [8]:
# Check for duplicates in movie_basics_df
movie_basics_df[movie_basics_df.duplicated(subset='movie_id')]

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres


In [9]:
# View the overall shape, dtypes and null counts for each column in movie_basics_df
movie_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [10]:
# Preview what is listed under the null genre entries
movie_basics_df[movie_basics_df['genres'].isna()]

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
16,tt0187902,How Huang Fei-hong Rescued the Orphan from the...,How Huang Fei-hong Rescued the Orphan from the...,2011,NaN,None
22,tt0253093,Gangavataran,Gangavataran,2018,134.0,None
35,tt0306058,Second Coming,Second Coming,2012,95.0,None
40,tt0326592,The Overnight,The Overnight,2010,88.0,None
44,tt0330811,Regret Not Speaking,Regret Not Speaking,2011,NaN,None
...,...,...,...,...,...,...
146088,tt9907396,Footloose in the Cotswolds - Part 1,Footloose in the Cotswolds - Part 1,2016,118.0,None
146089,tt9907608,Footloose in the Cotswolds - Part 2,Footloose in the Cotswolds - Part 2,2016,102.0,None
146107,tt9910922,Doctor Who Augmented Reality: Times Magazine,Doctor Who Augmented Reality: Times Magazine,2013,NaN,None
146129,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None


There are no null entries in the `movie_id` and `primary_title` columns. There are 5,408 nulls in the genres column. I am using this table specifically for genres, so I will be drops the columns where the genre is null.

In [11]:
# Preview first 5 entries of directors_df
directors_df.head()

,movie_id,person_id
0,tt0063540,nm0712540
1,tt0066787,nm0002411
2,tt0069049,nm0000080
3,tt0069204,nm0611531
4,tt0100275,nm0765384


In [12]:
# View the overall shape, dtypes and null counts for each column in directors_df
directors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140417 entries, 0 to 140416
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   140417 non-null  object
 1   person_id  140417 non-null  object
dtypes: object(2)
memory usage: 2.1+ MB


In [13]:
# Check for duplicates in directors_df
directors_df[directors_df.duplicated(subset='movie_id')]

,movie_id,person_id


There are no nulls in `directors_df` but there are many duplicates. If there are 

In [14]:
# Preview first 5 entries of persons_df
persons_df.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [15]:
# View the overall shape, dtypes and null counts for each column in persons_df
persons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


All I want from `persons_df` is the `person_id` and `primary_name` which neither have nulls. I don't need to handle any nulls here.

In [16]:
# Check for duplicates in persons_df
persons_df[persons_df.duplicated(subset='person_id')].head()

,person_id,primary_name,birth_year,death_year,primary_profession


### Rotten Tomatoes

In [17]:
# Preview first 5 entries of rt_movie_info_df
rt_movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [18]:
# View the overall shape, dtypes and null counts for each column in rt_move_info_df
rt_movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


This dataset looks limited. There are only 1560 entries and there are no movie titles, rather id numbers. I am not able to match the id's with any movie title. Once I determine which director to use, I will then return to this dataset to determine the genre of the movie. I don't want to use `box_office` to determine success in this dataset because there are only 340 non-null values.

In [38]:
# View the movie rating options and counts
rt_movie_info_df['rating'].value_counts(dropna=False)

rating
R        521
NR       503
PG       240
PG-13    235
G         57
NaN        3
NC17       1
Name: count, dtype: int64

There are only 3 null values in the `rating` column. They are listed as NaN so I believe it is safe to just drop them.

## Data Cleaning

### Box Office Mojo

In [101]:
# Keep columns title, domestic_gross, and foreign_gross
bom_df_clean = bom_df[['title', 'domestic_gross', 'foreign_gross']]

In [106]:
# Change nulls in foreign_gross and domestic_gross columns to $0
bom_df_clean['foreign_gross'] = bom_df_clean['foreign_gross'].fillna(0)
bom_df_clean['domestic_gross'] = bom_df_clean['domestic_gross'].fillna(0)

# Remove commas in dollar amounts in foreign_gross
bom_df_clean['foreign_gross'] = bom_df_clean['foreign_gross'].map(lambda x: str(x).replace(',',''))

# Change datatype in foreign_gross to float
bom_df_clean['foreign_gross'] = bom_df_clean['foreign_gross'].astype('float')

In [109]:
# Check changes to nulls and data types
bom_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3387 non-null   float64
 2   foreign_gross   3387 non-null   float64
dtypes: float64(2), object(1)
memory usage: 79.5+ KB


In [110]:
# Create total_gross column to include sum of domestic_gross and foreign_gross
bom_df_clean['total_gross'] = bom_df_clean['domestic_gross'] + bom_df_clean['foreign_gross']

# Preview changes
bom_df_clean.head()

,title,domestic_gross,foreign_gross,total_gross
0,Toy Story 3,415000000.0,652000000.0,1.067000e+09
1,Alice in Wonderland (2010),334200000.0,691300000.0,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000.0,9.603000e+08
3,Inception,292600000.0,535700000.0,8.283000e+08
4,Shrek Forever After,238700000.0,513900000.0,7.526000e+08


### IMDB

In [33]:
# Join the three tables into one dataframe
# Add all directors for one movie on the same line seperated by a comma and a space

query_imdb = """

SELECT mb.primary_title AS Movie_Title, mb.genres AS Genres, replace(GROUP_CONCAT(DISTINCT p.primary_name),',', ', ') AS Directors
  FROM movie_basics AS mb
  JOIN directors AS d
 USING (movie_id)
  JOIN persons AS p
 USING (person_id)
GROUP BY movie_id

"""

imdb_df = pd.read_sql(query_imdb, conn)

imdb_df.sample(20)


,Movie_Title,Genres,Directors
75290,Cancelled Faces,Drama,Lior Shamriz
10626,Fear of Flying,"Comedy,Romance",John Flores
92000,Zaplyv,Documentary,Kristina Paustian
42933,The Buffalo King,"Biography,Documentary,History",Justin Koehler
46189,Sodoma... L'altra faccia di Gomorra,Comedy,Vincenzo Pirozzi
100045,Thirty-Seventeen,Comedy,Miles Forster
98940,His Thoughts on the Interior,Documentary,Zach Crosswait
126563,Zhagaram,Drama,Krish
95312,Contro Corrente,"Documentary,Drama,Sport","Nicola D'Auria, Vincenzo Martone"
112210,Kim Swims,"Adventure,Biography,Documentary",Kate Webber


In [39]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140416 entries, 0 to 140415
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Movie_Title  140416 non-null  object
 1   Genres       137238 non-null  object
 2   Directors    140416 non-null  object
dtypes: object(3)
memory usage: 3.2+ MB


### Rotten Tomatoes

In [123]:
rt_movie_info_df_clean = rt_movie_info_df[['id', 'rating', 'director', 'box_office']]

In [128]:
# Drop nulls from 'rating' column
rt_movie_info_df_clean.dropna(subset='rating', inplace=True)

In [130]:
# Check nulls
rt_movie_info_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1557 entries, 0 to 1559
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1557 non-null   int64 
 1   rating      1557 non-null   object
 2   director    1361 non-null   object
 3   box_office  340 non-null    object
dtypes: int64(1), object(3)
memory usage: 60.8+ KB


In [131]:
# Preview changes
rt_movie_info_df_clean.head()

,id,rating,director,box_office
0,1,R,William Friedkin,NaN
1,3,R,David Cronenberg,"600,000"
2,5,R,Allison Anders,NaN
3,6,R,Barry Levinson,NaN
4,7,NR,Rodney Bennett,NaN


# Exploratory Data Analysis

# Conclusions

## Limitations

## Recommendations

## Next Steps